# Looking for obsolete terms without replacements

In this example, we use `fastobo` to create a small validation script which will retrieve obsolete terms without replacement.

In [1]:
import fastobo
import requests

`fastobo.load` takes a file-handle, which can be accessed using the `raw` property of the `Response` object returned by `requests.get`:

In [2]:
res = requests.get("http://purl.obolibrary.org/obo/go.obo", stream=True)
doc = fastobo.load(res.raw)

Note that we are using `isinstance` a lot compared to what you may be used to in other Python library: this is because `fastobo` is based on a Rust library which is strongly-typed, so that is reflected in the Python library that wraps it. We could use the strong typing to write the same snippet using type-specific callback wrapped in a `dict`:

In [3]:
for frame in doc:
    
    if isinstance(frame, fastobo.term.TermFrame):
    
        obsolete = False
        replacements = []

        for clause in frame:
            if clause.raw_tag == "is_obsolete":
                obsolete |= clause.obsolete
            elif clause.raw_tag in ("consider", "replaced_by"):
                replacements.append(clause.term)

        if obsolete and not replacements:
            print(frame.id, "is obsolete but has no replacement.")

Note that we could use the same kind of logic to retrieve terms with more than one replacement, which can be the case when an obsolete term does not have a strictly equivalent substitute in the newer versions of an ontology